# Tinh chỉnh các mô hình Open AI

Notebook này dựa trên hướng dẫn hiện tại được cung cấp trong tài liệu [Fine Tuning](https://platform.openai.com/docs/guides/fine-tuning?WT.mc_id=academic-105485-koreyst) của Open AI.

Tinh chỉnh giúp cải thiện hiệu suất của các mô hình nền tảng cho ứng dụng của bạn bằng cách huấn luyện lại với dữ liệu bổ sung và ngữ cảnh liên quan đến trường hợp sử dụng hoặc kịch bản cụ thể đó. Lưu ý rằng các kỹ thuật thiết kế prompt như _few shot learning_ và _retrieval augmented generation_ cho phép bạn tăng cường prompt mặc định với dữ liệu liên quan để nâng cao chất lượng. Tuy nhiên, các phương pháp này bị giới hạn bởi kích thước cửa sổ token tối đa của mô hình nền tảng được chọn.

Với tinh chỉnh, chúng ta thực chất đang huấn luyện lại chính mô hình với dữ liệu cần thiết (cho phép sử dụng nhiều ví dụ hơn so với giới hạn cửa sổ token tối đa) - và triển khai một phiên bản _tùy chỉnh_ của mô hình mà không còn cần cung cấp ví dụ khi suy luận nữa. Điều này không chỉ giúp tăng hiệu quả cho việc thiết kế prompt (chúng ta có nhiều linh hoạt hơn trong việc sử dụng cửa sổ token cho các mục đích khác) mà còn có thể giúp giảm chi phí (bằng cách giảm số lượng token cần gửi đến mô hình khi suy luận).

Tinh chỉnh gồm 4 bước:
1. Chuẩn bị dữ liệu huấn luyện và tải lên.
1. Chạy tác vụ huấn luyện để có được mô hình đã tinh chỉnh.
1. Đánh giá mô hình đã tinh chỉnh và lặp lại để nâng cao chất lượng.
1. Triển khai mô hình đã tinh chỉnh để suy luận khi đã hài lòng.

Lưu ý rằng không phải tất cả các mô hình nền tảng đều hỗ trợ tinh chỉnh - [xem tài liệu OpenAI](https://platform.openai.com/docs/guides/fine-tuning/what-models-can-be-fine-tuned?WT.mc_id=academic-105485-koreyst) để biết thông tin mới nhất. Bạn cũng có thể tinh chỉnh một mô hình đã được tinh chỉnh trước đó. Trong hướng dẫn này, chúng ta sẽ sử dụng `gpt-35-turbo` làm mô hình nền tảng mục tiêu để tinh chỉnh. 

---


### Bước 1.1: Chuẩn bị bộ dữ liệu của bạn

Hãy cùng xây dựng một chatbot giúp bạn hiểu về bảng tuần hoàn các nguyên tố bằng cách trả lời các câu hỏi về một nguyên tố dưới dạng thơ limerick. Trong _hướng dẫn_ đơn giản này, chúng ta chỉ tạo một bộ dữ liệu để huấn luyện mô hình với một vài ví dụ mẫu về các phản hồi, nhằm minh họa định dạng dữ liệu mong muốn. Trong trường hợp thực tế, bạn sẽ cần tạo một bộ dữ liệu với nhiều ví dụ hơn. Bạn cũng có thể sử dụng bộ dữ liệu mở (phù hợp với lĩnh vực ứng dụng của bạn) nếu có, và định dạng lại để sử dụng cho việc tinh chỉnh.

Vì chúng ta tập trung vào `gpt-35-turbo` và chỉ cần phản hồi một lượt (chat completion), chúng ta có thể tạo ví dụ theo [định dạng đề xuất này](https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset?WT.mc_id=academic-105485-koreyst) phù hợp với yêu cầu chat completion của OpenAI. Nếu bạn muốn nội dung hội thoại nhiều lượt, bạn sẽ dùng [định dạng ví dụ nhiều lượt](https://platform.openai.com/docs/guides/fine-tuning/multi-turn-chat-examples?WT.mc_id=academic-105485-koreyst) có thêm tham số `weight` để xác định tin nhắn nào nên dùng (hoặc không) trong quá trình tinh chỉnh.

Chúng ta sẽ dùng định dạng đơn giản một lượt cho hướng dẫn này. Dữ liệu ở [định dạng jsonl](https://jsonlines.org/?WT.mc_id=academic-105485-koreyst) với 1 bản ghi trên mỗi dòng, mỗi bản ghi là một đối tượng JSON. Đoạn mã dưới đây minh họa 2 bản ghi mẫu – xem [training-data.jsonl](../../../../../18-fine-tuning/python/openai/training-data.jsonl) để xem đầy đủ bộ ví dụ (10 ví dụ) mà chúng ta sẽ dùng cho hướng dẫn tinh chỉnh này. **Lưu ý:** Mỗi bản ghi _phải_ được viết trên một dòng duy nhất (không tách thành nhiều dòng như thường thấy ở file JSON định dạng).

```json
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Gallium"}, {"role": "assistant", "content": "Gallium, oh gallium, so light - Melts in your hand, oh what a sight - At 86 degrees - Its liquid with ease - And in semiconductors, it's out of sight"}]}
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Hydrogen"}, {"role": "assistant", "content": "Hydrogen, the first in the line - The lightest of all, so divine - It's in water, you see - And in stars, it's the key - The universe's most common sign"}]}
```

Trong thực tế, bạn sẽ cần một bộ ví dụ lớn hơn nhiều để đạt kết quả tốt – bạn sẽ phải cân nhắc giữa chất lượng phản hồi và thời gian/chi phí tinh chỉnh. Chúng ta dùng bộ nhỏ để có thể hoàn thành tinh chỉnh nhanh, minh họa quy trình. Xem [ví dụ OpenAI Cookbook này](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_finetune_chat_models.ipynb?WT.mc_id=academic-105485-koreyst) để tham khảo hướng dẫn tinh chỉnh phức tạp hơn.


### Bước 1.2 Tải lên Bộ Dữ Liệu của Bạn

Tải dữ liệu lên bằng cách sử dụng Files API [như được mô tả ở đây](https://platform.openai.com/docs/guides/fine-tuning/upload-a-training-file). Lưu ý rằng để chạy được đoạn mã này, bạn cần thực hiện các bước sau trước:
 - Đã cài đặt gói Python `openai` (hãy đảm bảo bạn sử dụng phiên bản >=0.28.0 để có các tính năng mới nhất)
 - Đã thiết lập biến môi trường `OPENAI_API_KEY` với khóa API OpenAI của bạn
Để tìm hiểu thêm, hãy xem [Hướng dẫn cài đặt](./../../../00-course-setup/02-setup-local.md?WT.mc_id=academic-105485-koreyst) được cung cấp cho khóa học.

Bây giờ, hãy chạy đoạn mã để tạo một tệp từ file JSONL trên máy của bạn để tải lên.


In [24]:
from openai import OpenAI
client = OpenAI()

ft_file = client.files.create(
  file=open("./training-data.jsonl", "rb"),
  purpose="fine-tune"
)

print(ft_file)
print("Training File ID: " + ft_file.id)

FileObject(id='file-JdAJcagdOTG6ACNlFWzuzmyV', bytes=4021, created_at=1715566183, filename='training-data.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)
Training File ID: file-JdAJcagdOTG6ACNlFWzuzmyV


### Bước 2.1: Tạo job Fine-tuning với SDK


In [25]:
from openai import OpenAI
client = OpenAI()

ft_filejob = client.fine_tuning.jobs.create(
  training_file=ft_file.id, 
  model="gpt-3.5-turbo"
)

print(ft_filejob)
print("Fine-tuning Job ID: " + ft_filejob.id)

FineTuningJob(id='ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', created_at=1715566184, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-EZ6ag0n0S6Zm8eV9BSWKmE6l', result_files=[], seed=830529052, status='validating_files', trained_tokens=None, training_file='file-JdAJcagdOTG6ACNlFWzuzmyV', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)
Fine-tuning Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh


### Bước 2.2: Kiểm tra trạng thái của job

Dưới đây là một số thao tác bạn có thể thực hiện với API `client.fine_tuning.jobs`:
- `client.fine_tuning.jobs.list(limit=<n>)` - Liệt kê n job fine-tuning gần nhất
- `client.fine_tuning.jobs.retrieve(<job_id>)` - Lấy thông tin chi tiết của một job fine-tuning cụ thể
- `client.fine_tuning.jobs.cancel(<job_id>)` - Hủy một job fine-tuning
- `client.fine_tuning.jobs.list_events(fine_tuning_job_id=<job_id>, limit=<b>)` - Liệt kê tối đa n sự kiện từ job
- `client.fine_tuning.jobs.create(model="gpt-35-turbo", training_file="your-training-file.jsonl", ...)`

Bước đầu tiên của quá trình là _kiểm tra tính hợp lệ của file huấn luyện_ để đảm bảo dữ liệu có đúng định dạng.


In [26]:
from openai import OpenAI
client = OpenAI()

# List 10 fine-tuning jobs
client.fine_tuning.jobs.list(limit=10)

# Retrieve the state of a fine-tune
client.fine_tuning.jobs.retrieve(ft_filejob.id)

# List up to 10 events from a fine-tuning job
client.fine_tuning.jobs.list_events(fine_tuning_job_id=ft_filejob.id, limit=10)

SyncCursorPage[FineTuningJobEvent](data=[FineTuningJobEvent(id='ftevent-GkWiDgZmOsuv4q5cSTEGscY6', created_at=1715566184, level='info', message='Validating training file: file-JdAJcagdOTG6ACNlFWzuzmyV', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-3899xdVTO3LN7Q7LkKLMJUnb', created_at=1715566184, level='info', message='Created fine-tuning job: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', object='fine_tuning.job.event', data={}, type='message')], object='list', has_more=False)

In [30]:
# Once the training data is validated
# Track the job status to see if it is running and when it is complete
from openai import OpenAI
client = OpenAI()

response = client.fine_tuning.jobs.retrieve(ft_filejob.id)

print("Job ID:", response.id)
print("Status:", response.status)
print("Trained Tokens:", response.trained_tokens)

Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh
Status: running
Trained Tokens: None


### Bước 2.3: Theo dõi sự kiện để giám sát tiến trình


In [44]:
# You can also track progress in a more granular way by checking for events
# Refresh this code till you get the `The job has successfully completed` message
response = client.fine_tuning.jobs.list_events(ft_filejob.id)

events = response.data
events.reverse()

for event in events:
    print(event.message)

Step 85/100: training loss=0.14
Step 86/100: training loss=0.00
Step 87/100: training loss=0.00
Step 88/100: training loss=0.07
Step 89/100: training loss=0.00
Step 90/100: training loss=0.00
Step 91/100: training loss=0.00
Step 92/100: training loss=0.00
Step 93/100: training loss=0.00
Step 94/100: training loss=0.00
Step 95/100: training loss=0.08
Step 96/100: training loss=0.05
Step 97/100: training loss=0.00
Step 98/100: training loss=0.00
Step 99/100: training loss=0.00
Step 100/100: training loss=0.00
Checkpoint created at step 80 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyyF2:ckpt-step-80
Checkpoint created at step 90 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyzhK:ckpt-step-90
New fine-tuned model created: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz
The job has successfully completed


### Bước 2.4: Xem trạng thái trong Bảng điều khiển OpenAI


Bạn cũng có thể xem trạng thái bằng cách truy cập trang web OpenAI và khám phá phần _Fine-tuning_ trên nền tảng. Tại đây, bạn sẽ thấy trạng thái của công việc hiện tại, đồng thời có thể theo dõi lịch sử các lần chạy trước đó. Trong ảnh chụp màn hình này, bạn có thể thấy lần chạy trước đã thất bại, còn lần chạy thứ hai thì thành công. Để bạn dễ hình dung, điều này xảy ra khi lần chạy đầu tiên sử dụng một tệp JSON với các bản ghi bị định dạng sai - sau khi sửa lại, lần chạy thứ hai đã hoàn tất thành công và mô hình đã sẵn sàng để sử dụng.

![Fine-tuning job status](../../../../../translated_images/fine-tuned-model-status.563271727bf7bfba7e3f73a201f8712fae3cea1c08f7c7f12ca469c06d234122.vi.png)


Bạn cũng có thể xem các thông báo trạng thái và số liệu bằng cách cuộn xuống phía dưới trong bảng điều khiển trực quan như hình minh họa:

| Thông báo | Số liệu |
|:---|:---|
| ![Messages](../../../../../translated_images/fine-tuned-messages-panel.4ed0c2da5ea1313b3a706a66f66bf5007c379cd9219cfb74cb30c0b04b90c4c8.vi.png) |  ![Metrics](../../../../../translated_images/fine-tuned-metrics-panel.700d7e4995a652299584ab181536a6cfb67691a897a518b6c7a2aa0a17f1a30d.vi.png)|


### Bước 3.1: Lấy ID & Kiểm tra mô hình đã tinh chỉnh trong mã nguồn


In [46]:
# Retrieve the identity of the fine-tuned model once ready
response = client.fine_tuning.jobs.retrieve(ft_filejob.id)
fine_tuned_model_id = response.fine_tuned_model
print("Fine-tuned Model ID:", fine_tuned_model_id)

Fine-tuned Model ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz


In [47]:
# You can then use that model to generate completions from the SDK as shown
# Or you can load that model into the OpenAI Playground (in the UI) to validate it from there.
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
  model=fine_tuned_model_id,
  messages=[
    {"role": "system", "content": "You are Elle, a factual chatbot that answers questions about elements in the periodic table with a limerick"},
    {"role": "user", "content": "Tell me about Strontium"},
  ]
)
print(completion.choices[0].message)

ChatCompletionMessage(content="Strontium, a metal so bright - It's in fireworks, a dazzling sight - It's in bones, you see - And in tea, it's the key - It's the fortieth, so pure, that's the right", role='assistant', function_call=None, tool_calls=None)


### Bước 3.2: Tải & Kiểm tra Mô hình Đã Tinh chỉnh trong Playground

Bây giờ bạn có thể kiểm tra mô hình đã tinh chỉnh theo hai cách. Đầu tiên, bạn có thể truy cập Playground và sử dụng menu thả xuống Models để chọn mô hình vừa tinh chỉnh từ các lựa chọn có sẵn. Cách thứ hai là sử dụng tùy chọn "Playground" hiển thị trong bảng điều khiển Fine-tuning (xem ảnh chụp màn hình phía trên), sẽ mở ra giao diện _so sánh_ cho phép bạn xem song song phiên bản mô hình gốc và mô hình đã tinh chỉnh để đánh giá nhanh.

![Fine-tuning job status](../../../../../translated_images/fine-tuned-playground-compare.56e06f0ad8922016497d39ced3d84ea296eec89073503f2bf346ec9718f913b5.vi.png)

Chỉ cần điền vào phần ngữ cảnh hệ thống đã dùng trong dữ liệu huấn luyện và nhập câu hỏi kiểm tra của bạn. Bạn sẽ thấy cả hai bên đều được cập nhật với cùng một ngữ cảnh và câu hỏi. Chạy so sánh và bạn sẽ thấy sự khác biệt trong kết quả giữa hai mô hình. _Lưu ý cách mô hình đã tinh chỉnh trả lời theo đúng định dạng bạn đã cung cấp trong các ví dụ, trong khi mô hình gốc chỉ đơn giản làm theo hướng dẫn hệ thống_.

![Fine-tuning job status](../../../../../translated_images/fine-tuned-playground-launch.5a26495c983c6350c227e05700a47a89002d132949a56fa4ff37f266ebe997b2.vi.png)

Bạn cũng sẽ thấy phần so sánh hiển thị số lượng token của mỗi mô hình, cùng với thời gian thực hiện suy luận. **Ví dụ cụ thể này chỉ mang tính minh họa quy trình, không phản ánh một bộ dữ liệu hay tình huống thực tế**. Có thể bạn sẽ nhận thấy cả hai mẫu đều có số lượng token giống nhau (ngữ cảnh hệ thống và câu hỏi người dùng đều giống), nhưng mô hình đã tinh chỉnh mất nhiều thời gian hơn để suy luận (do là mô hình tùy chỉnh).

Trong thực tế, bạn sẽ không dùng ví dụ đơn giản như thế này, mà sẽ tinh chỉnh dựa trên dữ liệu thực (ví dụ: danh mục sản phẩm cho dịch vụ khách hàng) nơi chất lượng phản hồi sẽ rõ ràng hơn nhiều. Trong _bối cảnh đó_, để đạt được chất lượng phản hồi tương đương với mô hình gốc, bạn sẽ cần phải thiết kế prompt tùy chỉnh phức tạp hơn, dẫn đến sử dụng nhiều token hơn và có thể tăng thời gian xử lý khi suy luận. _Để thử nghiệm, bạn có thể tham khảo các ví dụ tinh chỉnh trong OpenAI Cookbook để bắt đầu._



---

**Tuyên bố miễn trừ trách nhiệm**:  
Tài liệu này đã được dịch bằng dịch vụ dịch thuật AI [Co-op Translator](https://github.com/Azure/co-op-translator). Mặc dù chúng tôi cố gắng đảm bảo độ chính xác, xin lưu ý rằng các bản dịch tự động có thể chứa lỗi hoặc không chính xác. Tài liệu gốc bằng ngôn ngữ bản địa nên được coi là nguồn tham khảo chính thức. Đối với các thông tin quan trọng, nên sử dụng dịch vụ dịch thuật chuyên nghiệp bởi con người. Chúng tôi không chịu trách nhiệm đối với bất kỳ sự hiểu lầm hoặc diễn giải sai nào phát sinh từ việc sử dụng bản dịch này.
